<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [89]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [90]:
with open('./datasets/borges_full_texts.pkl', 'rb') as f:
   borges = pickle.load(f)

In [91]:
borges.head()

,link,text_metadata,text
0,https://ciudadseva.com/texto/abel-y-cain-borges/,"{'title': 'Abel y Caín', 'metadata': '[Minicue...",Abel y Caín se encontraron después de la muert...
1,https://ciudadseva.com/texto/adrogue/,"{'title': 'Adrogué', 'metadata': '[Minicuento ...","Era muy lindo, un pueblo laberíntico. A veces,..."
2,https://ciudadseva.com/texto/alguien-sonara/,"{'title': 'Alguien soñará', 'metadata': '[Mini...",¿Qué soñará el indescifrable futuro? Soñará qu...
3,https://ciudadseva.com/texto/andres-armoa/,"{'title': 'Andrés Armoa', 'metadata': '[Minicu...",Los años le han dejado unas palabras en guaran...
4,https://ciudadseva.com/texto/argumentum-ornith...,"{'title': 'Argumentum ornithologicum', 'metada...",Cierro los ojos y veo una bandada de pájaros. ...


In [92]:
borges['text'][1]

'Era muy lindo, un pueblo laberíntico. A veces, algunas noches de verano, salíamos mi padre, mi madre y yo a perdernos. Al principio nos costaba un poco de trabajo, pero luego nos perfeccionamos tanto que nos perdíamos enseguida. FIN'

In [93]:
text = borges["text"]
df = pd.DataFrame(text)
type(df)

pandas.core.frame.DataFrame

In [94]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 62


### 1 - Preprocesamiento

In [95]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [96]:
# Demos un vistazo
sentence_tokens[:2]

[['abel',
  'y',
  'caín',
  'se',
  'encontraron',
  'después',
  'de',
  'la',
  'muerte',
  'de',
  'abel',
  'caminaban',
  'por',
  'el',
  'desierto',
  'y',
  'se',
  'reconocieron',
  'desde',
  'lejos',
  'porque',
  'los',
  'dos',
  'eran',
  'muy',
  'altos',
  'los',
  'hermanos',
  'se',
  'sentaron',
  'en',
  'la',
  'tierra',
  'hicieron',
  'un',
  'fuego',
  'y',
  'comieron',
  'guardaban',
  'silencio',
  'a',
  'la',
  'manera',
  'de',
  'la',
  'gente',
  'cansada',
  'cuando',
  'declina',
  'el',
  'día',
  'en',
  'el',
  'cielo',
  'asomaba',
  'alguna',
  'estrella',
  'que',
  'aún',
  'no',
  'había',
  'recibido',
  'su',
  'nombre',
  'a',
  'la',
  'luz',
  'de',
  'las',
  'llamas',
  'caín',
  'advirtió',
  'en',
  'la',
  'frente',
  'de',
  'abel',
  'la',
  'marca',
  'de',
  'la',
  'piedra',
  'y',
  'dejó',
  'caer',
  'el',
  'pan',
  'que',
  'estaba',
  'por',
  'llevarse',
  'a',
  'la',
  'boca',
  'y',
  'pidió',
  'que',
  'le',
  'fuera

### 2 - Crear los vectores (word2vec)

In [97]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [98]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=250,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [99]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [100]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 62


In [101]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 2016


In [102]:
w2v_model.wv.index_to_key[1000]

'abril'

In [103]:
w2v_model.wv.index_to_key[1000:1015]

['abril',
 'nueva',
 'borracho',
 'n',
 'tabares',
 'natal',
 'enero',
 'suárez',
 'sorteo',
 'juliana',
 'tercero',
 'isidoro',
 'tercera',
 'soldado',
 'poeta']

### 3 - Entrenar embeddings

In [104]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 675937.25
Loss after epoch 1: 489500.75
Loss after epoch 2: 441047.125
Loss after epoch 3: 439871.625
Loss after epoch 4: 385788.0
Loss after epoch 5: 376092.25
Loss after epoch 6: 374212.0
Loss after epoch 7: 373471.75
Loss after epoch 8: 371190.75
Loss after epoch 9: 365899.0
Loss after epoch 10: 353964.0
Loss after epoch 11: 350131.0
Loss after epoch 12: 350685.0
Loss after epoch 13: 348183.0
Loss after epoch 14: 347845.0
Loss after epoch 15: 346488.5
Loss after epoch 16: 345849.5
Loss after epoch 17: 344842.0
Loss after epoch 18: 343486.5
Loss after epoch 19: 344116.5


(828944, 1743720)

### 4 - Ensayar

In [105]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["pueblo"], topn=10)

[('notoriamente', 0.9683837294578552),
 ('schopenhauer', 0.9601207375526428),
 ('suertes', 0.9543336033821106),
 ('soñador', 0.9538471698760986),
 ('declaró', 0.9531726837158203),
 ('hipótesis', 0.9524017572402954),
 ('abandonó', 0.9522297978401184),
 ('atributo', 0.9520503878593445),
 ('curso', 0.9492930769920349),
 ('heresiarca', 0.9477555751800537)]

In [106]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["borracho"], topn=10)

[('sin', -0.3199578523635864),
 ('primera', -0.345326691865921),
 ('al', -0.34675565361976624),
 ('ese', -0.36129897832870483),
 ('primer', -0.3634583353996277),
 ('cada', -0.36355316638946533),
 ('he', -0.3651367127895355),
 ('sus', -0.3703135550022125),
 ('esa', -0.371721088886261),
 ('del', -0.37495285272598267)]

In [107]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["borracho"], topn=10)

[('entendimiento', 0.9300537109375),
 ('siguió', 0.9297990202903748),
 ('forastero', 0.9295588731765747),
 ('aceptó', 0.9204953908920288),
 ('parecido', 0.9195415377616882),
 ('oyera', 0.9168623089790344),
 ('sonrisa', 0.9155243635177612),
 ('comer', 0.9149011969566345),
 ('cristo', 0.9145134091377258),
 ('gaucho', 0.91443932056427)]

In [108]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["poeta"], topn=5)

[('varón', 0.941706120967865),
 ('espíritu', 0.9392508268356323),
 ('oído', 0.9319267272949219),
 ('posterior', 0.9276713728904724),
 ('error', 0.9258841872215271)]

In [109]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["oso"])

KeyError: "Key 'oso' not present in vocabulary"

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_futuro = w2v_model.wv.get_vector("futuro")
print(vector_futuro)

[ 0.19832844 -0.03587237  0.00056125  0.02111164 -0.03050752  0.01417337
 -0.15274483  0.1073123   0.1394657   0.0699709  -0.07581188  0.07899535
 -0.13054664  0.10077302  0.05476725 -0.1448277  -0.14999127  0.00543945
  0.08177578  0.01407057  0.02128148  0.02327199  0.0916009  -0.09134521
 -0.00194111 -0.12239401  0.10127061 -0.08233527 -0.03302703  0.10678101
  0.05849391  0.12256519 -0.07563865 -0.06970628  0.17548193  0.07286508
 -0.05837394 -0.01146768  0.16686122 -0.09663258 -0.04731658  0.13864727
 -0.02432273  0.04745548 -0.10224677 -0.04894228 -0.07956331 -0.00450605
  0.05754686 -0.06541813 -0.21652858  0.03890591  0.11002529 -0.16392061
  0.22605018 -0.16155355  0.00948208  0.10669906  0.02194656 -0.18366674
 -0.05648258 -0.05413198  0.05627433 -0.04128056  0.01403201  0.05957552
  0.07190096 -0.17336477  0.01334898 -0.09120337  0.03286209 -0.13874553
 -0.08720814 -0.1703112   0.05991164  0.10519896 -0.25290167 -0.03896395
  0.06550569  0.1282792  -0.00620425  0.2015469  -0

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_futuro)

[('futuro', 0.9999999403953552),
 ('indiferente', 0.9489802122116089),
 ('local', 0.9475724697113037),
 ('decente', 0.9464311599731445),
 ('ayer', 0.9429251551628113),
 ('inmediato', 0.9412068128585815),
 ('narrador', 0.9411382675170898),
 ('atributo', 0.940750241279602),
 ('negar', 0.939579963684082),
 ('ley', 0.939224362373352)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["casa"], topn=10)

[('quinta', 0.8296251893043518),
 ('pieza', 0.806225061416626),
 ('cabaña', 0.8043193817138672),
 ('inglesa', 0.7969458699226379),
 ('vuelta', 0.7911803126335144),
 ('cayó', 0.7843554615974426),
 ('hermano', 0.7842481732368469),
 ('rancho', 0.7806351184844971),
 ('garay', 0.7787056565284729),
 ('oscuridad', 0.7764340043067932)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
#fig.show(renderer="colab") # esto para plotly en colab

In [110]:
vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])

Oservación:  
Los vectores de las palabras "mujer" y "casa" son similares. Eso significa que aparecen en el mismo contexto.

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
#fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.